### ID2214/FID3214 Assignment 3 Group no. 12

Project members:

Aksel Uhr, auhr@kth.se 
Olivia Höft, hoft@kth.se 
Ilias Merentitis, iliasme@kth.se

Declaration

By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.
It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from rdkit import Chem
import rdkit.Chem.rdMolDescriptors as d
import rdkit.Chem.Descriptors as d2
import rdkit.Chem.Fragments as f
import rdkit.Chem.Lipinski as l
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer

In [ ]:
train_df = pd.read_csv("training_smiles.csv")
test_df = pd.read_csv("test_smiles.csv")
seed = 10

In [ ]:
def feature_selection(train):
    dfCopy2 = train.copy()
    
    dfCopy2=dfCopy2.drop(labels=['INDEX'], axis=1)
    labels = dfCopy2["ACTIVE"]
    dfCopy2 = dfCopy2.drop(columns = ["ACTIVE"])
    
    dfCopy2['SMILES'] = dfCopy2['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))
    dfCopy2['NumAtoms'] = dfCopy2['SMILES'].apply(lambda x: x.GetNumAtoms())
    dfCopy2['HeavyAtomCount']=dfCopy2['SMILES'].apply(lambda x: l.HeavyAtomCount(x))
    dfCopy2['CalcExactMolWt']=dfCopy2['SMILES'].apply(lambda x: d.CalcExactMolWt(x))
    dfCopy2['fr_Al_COO']=dfCopy2['SMILES'].apply(lambda x: f.fr_Al_COO(x))
    dfCopy2['HsNumAtoms'] = dfCopy2["SMILES"].apply(lambda x: Chem.AddHs(x).GetNumAtoms())
    
    morganFingerPrint=[np.array(AllChem.GetMorganFingerprintAsBitVect(x,2,nBits=124)) for x in dfCopy2['SMILES']]
    morganFingerPrint=pd.DataFrame(morganFingerPrint)
    morganFingerPrint.columns=['fp_'+str(x) for x in morganFingerPrint.columns]    
    dfCopy2 = pd.concat([dfCopy2, morganFingerPrint], axis=1, join='inner')
    dfCopy2=dfCopy2.drop(labels=['SMILES'], axis=1)
    
    return dfCopy2, labels

In [ ]:
#feature_selection()
df, y_res = feature_selection(train_df)

In [ ]:
#Split the data
X_train, X_test, y_train, y_test = train_test_split(df, y_res, test_size=0.2, stratify=y_res) 

In [ ]:
#Make copies of the dataframes
X_trainCopy = X_train.copy()
y_trainCopy = y_train.copy()
X_testCopy = X_test.copy()
y_testCopy = y_test.copy()

In [ ]:
#Plot Histograms of selected features
withoutMorgan = X_trainCopy.iloc[:, :5]
withoutMorgan.hist(figsize=(12,12), xrot=45)
plt.show()

In [ ]:
# Correlation between features
corrs = withoutMorgan.corr()
display(corrs)

# Heatmap of correlations
plt.figure(figsize=(10,8))
sns.heatmap(corrs, cmap='RdBu_r', annot=True)
plt.show()

#Delete the correlatedFeatures
X_trainCopy = X_trainCopy.drop(columns = ["HeavyAtomCount", "CalcExactMolWt", "HsNumAtoms"])
X_testCopy = X_testCopy.drop(columns = ["HeavyAtomCount", "CalcExactMolWt", "HsNumAtoms"])
withoutMorgan = withoutMorgan.drop(columns = ["HeavyAtomCount", "CalcExactMolWt", "HsNumAtoms"])

# Correlation between features
corrs = withoutMorgan.corr()
display(corrs)

# Heatmap of correlations
plt.figure(figsize=(10,8))
sns.heatmap(corrs, cmap='RdBu_r', annot=True)
plt.show()

In [ ]:
print("Num of NaN in train: ", X_trainCopy.isna().sum().sum())
print("Training data shape: ", X_trainCopy.shape)
print("testing data shape:  ", X_testCopy.shape)
X_trainCopy.head(5)

In [ ]:
def create_pre_processing(train):
    imp = SimpleImputer(strategy="mean")
    imp.fit(train)
    imputatedDf = imp.transform(train)
    binning = KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="uniform")
    binning.fit(imputatedDf)
    binnedDf = binning.transform(imputatedDf)
    return binnedDf, imp, binning

In [ ]:
def apply_pre_processing(test, imputation, binning):
    imputation.fit(test)
    test = imputation.transform(test)
    binning.fit(test)
    test = binning.transform(test)
    return test

In [ ]:
#Cross validation to choose model

cv = StratifiedKFold(n_splits=10, shuffle=True)

gnb = GaussianNB()
pipelinegnb= make_pipeline(SimpleImputer(strategy='mean'), KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="uniform"), gnb)
scores_gnb = cross_val_score(pipelinegnb, X_trainCopy, y_trainCopy, scoring='roc_auc', cv=cv, n_jobs=-1)

rf = RandomForestClassifier()
pipelinerf= make_pipeline(SimpleImputer(strategy='mean'), KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="uniform"), rf)
scores_rf = cross_val_score(pipelinerf, X_trainCopy, y_trainCopy, scoring='roc_auc', cv=cv, n_jobs=-1)

In [ ]:
plt.boxplot([scores_rf, scores_gnb])
# NB
print('Mean ROC AUC for GaussianNB: %.3f' % mean(scores_gnb))

# Random forest
print('Mean ROC AUC for RandomForestClassifier: %.3f' % mean(scores_rf))

In [ ]:
# Train the choosen model

X_trainCopy, imp_dict, bin_dict =  create_pre_processing(X_trainCopy)

clf = BalancedRandomForestClassifier(random_state=seed)

param = {
    'n_estimators': [10, 20, 30],
    'max_features' : ['auto', 'sqrt', 'log2'],
    'max_depth' : [3, 5, 7], 
    'criterion' :['gini', 'entropy']
}

CV_clf = GridSearchCV(estimator=clf, param_grid=param, n_jobs = -1)

CV_clf.fit(X_trainCopy, y_trainCopy)

print("best param: ", CV_clf.best_params_)

In [ ]:
#Test the model on the test set
X_testCopy = apply_pre_processing(X_testCopy, imp_dict, bin_dict)

y_pred_proba = CV_clf.predict_proba(X_testCopy)
y_pred_proba = y_pred_proba[:, 1:]
print("y_pred_proba: ", y_pred_proba)

y_pred = CV_clf.predict(X_testCopy)
print("yPred: ", y_pred)

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
auc = metrics.auc(fpr, tpr)
print("auc: ", auc)

plt.figure(figsize=(12,6))
plt.plot(fpr, tpr,label='AUC')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random')
plt.title('AUC:{}'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.legend()

### Part2, More features


In [ ]:
def feature_selection2(train):
    dfCopy2 = train.copy()
    
    dfCopy2=dfCopy2.drop(labels=['INDEX'], axis=1)
    labels = dfCopy2["ACTIVE"]
    dfCopy2 = dfCopy2.drop(columns = ["ACTIVE"])

    dfCopy2['SMILES'] = dfCopy2['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))
    dfCopy2['fr_Al_COO']=dfCopy2['SMILES'].apply(lambda x: f.fr_Al_COO(x))
    dfCopy2['NOCount']=dfCopy2['SMILES'].apply(lambda x: l.NOCount(x))
    dfCopy2['NUM_H_ACCEPTORS']=dfCopy2['SMILES'].apply(lambda x: l.NumHAcceptors(x))
    dfCopy2['NUM_H_DONORS']=dfCopy2['SMILES'].apply(lambda x: l.NumHDonors(x))
    dfCopy2["NUM_OF_ATOMS"] = dfCopy2['SMILES'].apply(lambda x: x.GetNumAtoms())
    dfCopy2["DENSITY"] = dfCopy2.apply(lambda x: (d2.FpDensityMorgan1(x["SMILES"])), axis=1)    
    dfCopy2["MAX_PCHARGE"] = dfCopy2.apply(lambda x: (d2.MaxPartialCharge(x["SMILES"])), axis=1) 
    dfCopy2["MIN_PCHARGE"] = dfCopy2.apply(lambda x: (d2.MinPartialCharge(x["SMILES"])), axis=1) 
    dfCopy2["MOLDESCRIPTOR"] = dfCopy2.apply(lambda x: (d.CalcNumRings(x["SMILES"])), axis=1)   

    morganFingerPrint=[np.array(AllChem.GetMorganFingerprintAsBitVect(x,2,nBits=124)) for x in dfCopy2['SMILES']]
    morganFingerPrint=pd.DataFrame(morganFingerPrint)
    morganFingerPrint.columns=['fp_'+str(x) for x in morganFingerPrint.columns]    
    dfCopy2 = pd.concat([dfCopy2, morganFingerPrint], axis=1, join='inner')
    dfCopy2=dfCopy2.drop(labels=['SMILES'], axis=1)
    
    return dfCopy2, labels

In [ ]:
df, y_res = feature_selection2(train_df)

In [ ]:
#Split the data
X_train, X_test, y_train, y_test = train_test_split(df, y_res, test_size=0.2, stratify=y_res)   

In [ ]:
#Make copies of the dataframes
X_trainCopy = X_train.copy()
y_trainCopy = y_train.copy()
X_testCopy = X_test.copy()
y_testCopy = y_test.copy()

In [ ]:
#Plot Histograms of selected features
withoutMorgan = X_trainCopy.iloc[:, :12]
withoutMorgan.hist(figsize=(12,12), xrot=45)
plt.show()

In [ ]:
# Correlation between features
corrs = withoutMorgan.corr()
display(corrs)

# Heatmap of correlations
plt.figure(figsize=(10,8))
sns.heatmap(corrs, cmap='RdBu_r', annot=True)
plt.show()

In [ ]:
print("Num of NaN in train: ", X_trainCopy.isna().sum().sum())
print("Training data shape: ", X_trainCopy.shape)
print("testing data shape:  ", X_testCopy.shape)
X_trainCopy.head(5)

In [ ]:
#Cross validation to choose model

cv = StratifiedKFold(n_splits=10, shuffle=True)

gnb = GaussianNB()
pipelinegnb= make_pipeline(SimpleImputer(strategy='mean'), KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="uniform"), gnb)
scores_gnb = cross_val_score(pipelinegnb, X_trainCopy, y_trainCopy, scoring='roc_auc', cv=cv, n_jobs=-1)

rf = RandomForestClassifier()
pipelinerf= make_pipeline(SimpleImputer(strategy='mean'), KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="uniform"), rf)
scores_rf = cross_val_score(pipelinerf, X_trainCopy, y_trainCopy, scoring='roc_auc', cv=cv, n_jobs=-1)

In [ ]:
plt.boxplot([scores_rf, scores_gnb])
# NB
print('Mean ROC AUC for GaussianNB: %.3f' % mean(scores_gnb))

# Random forest
print('Mean ROC AUC for RandomForestClassifier: %.3f' % mean(scores_rf))

In [ ]:
# Train the choosen model

X_trainCopy, imp_dict, bin_dict=  create_pre_processing(X_trainCopy)

clf = BalancedRandomForestClassifier(random_state=seed)

param = {
    'n_estimators': [10, 20, 30],
    'max_features' : ['auto', 'sqrt', 'log2'],
    'max_depth' : [3, 5, 7],
    'criterion' :['gini', 'entropy']
}

CV_clf = GridSearchCV(estimator=clf, param_grid=param, n_jobs = -1, scoring = 'roc_auc')

CV_clf.fit(X_trainCopy, y_trainCopy)

print("best param: ", CV_clf.best_params_)

In [ ]:
#Test the model on the test set
X_testCopy = apply_pre_processing(X_testCopy, imp_dict, bin_dict)

y_pred_proba = CV_clf.predict_proba(X_testCopy)
y_pred_proba = y_pred_proba[:, 1:]
print("y_pred_proba: ", y_pred_proba)

y_pred = CV_clf.predict(X_testCopy)
print("yPred: ", y_pred)

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
auc = metrics.auc(fpr, tpr)
print("auc: ", auc)

plt.figure(figsize=(12,6))
plt.plot(fpr, tpr,label='AUC')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random')
plt.title('AUC:{}'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.legend()

### MAKE PREDICTION ON TEST FILE

In [ ]:
def feature_selection3(test):
    dfCopy2 = test.copy()
    
    dfCopy2=dfCopy2.drop(labels=['INDEX'], axis=1)

    dfCopy2['SMILES'] = dfCopy2['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))
    dfCopy2['fr_Al_COO']=dfCopy2['SMILES'].apply(lambda x: f.fr_Al_COO(x))
    dfCopy2['NOCount']=dfCopy2['SMILES'].apply(lambda x: l.NOCount(x))
    dfCopy2['NUM_H_ACCEPTORS']=dfCopy2['SMILES'].apply(lambda x: l.NumHAcceptors(x))
    dfCopy2['NUM_H_DONORS']=dfCopy2['SMILES'].apply(lambda x: l.NumHDonors(x))
    dfCopy2["NUM_OF_ATOMS"] = dfCopy2['SMILES'].apply(lambda x: x.GetNumAtoms())
    dfCopy2["DENSITY"] = dfCopy2.apply(lambda x: (d2.FpDensityMorgan1(x["SMILES"])), axis=1)    
    dfCopy2["MAX_PCHARGE"] = dfCopy2.apply(lambda x: (d2.MaxPartialCharge(x["SMILES"])), axis=1) 
    dfCopy2["MIN_PCHARGE"] = dfCopy2.apply(lambda x: (d2.MinPartialCharge(x["SMILES"])), axis=1) 
    dfCopy2["MOLDESCRIPTOR"] = dfCopy2.apply(lambda x: (d.CalcNumRings(x["SMILES"])), axis=1)  

    morganFingerPrint=[np.array(AllChem.GetMorganFingerprintAsBitVect(x,2,nBits=124)) for x in dfCopy2['SMILES']]
    morganFingerPrint=pd.DataFrame(morganFingerPrint)
    morganFingerPrint.columns=['fp_'+str(x) for x in morganFingerPrint.columns]    
    dfCopy2 = pd.concat([dfCopy2, morganFingerPrint], axis=1, join='inner')
    dfCopy2=dfCopy2.drop(labels=['SMILES'], axis=1)
    
    return dfCopy2

In [ ]:
#Predict on the original test file.

test_df_copy = test_df.copy()
test_df_copy = feature_selection3(test_df_copy)

test_df_copy = apply_pre_processing(test_df_copy, imp_dict, bin_dict)

y_pred_proba_test = CV_clf.predict_proba(test_df_copy)
y_pred_proba_test = y_pred_proba_test[:, 1:]
print("y_pred_proba: ", y_pred_proba_test)

y_pred_test = CV_clf.predict(test_df_copy)
print("yPred: ", y_pred_test)

In [ ]:
f = open('12.txt', 'w')
np.savetxt(f, y_pred_proba_test, newline='\n')
f.close() 